In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast

In [2]:
## read the tsv files in MovieSummaries
load_movies = pd.read_csv('MovieSummaries/movie.metadata.tsv', sep = '\t', header = None)
load_character = pd.read_csv('MovieSummaries/character.metadata.tsv', sep = '\t', header = None)



In [3]:
movies = load_movies.copy()
character = load_character.copy()


In [4]:
# Make the name of the colum of movie: wikip_ID, freebase_ID, name, release date, box office, runtime, languages, countries, genres	
movies.columns = ['Wiki_ID', 'Freebase_ID', 'name', 'release_date', 'box_office', 'runtime', 'languages', 'countries', 'genres']
# Make the name of the columns of character wiki_ID, freebase_ID, character_name, actor_DOB, actor_gender, actor_height, actor_ethnicity, actor_name, actor_age, freebase_character_map
character.columns = ['Wiki_ID', 'Freebase_ID','release_date', 'character_name', 'actor_DOB', 'actor_gender', 'actor_height', 'actor_ethnicity', 'actor_name', 'actor_age', '10', '11', '12']
character = character.drop(['10', '11', '12'], axis = 1)

In [5]:
## read grossed merged clean csv
gross = pd.read_csv('gross_merged_clean.csv')
gross

,tconst,Wiki_ID,Freebase_ID,country,movie,gross
0,tt0228333,975900,/m/03vyhn,Domestic,ghosts of mars,8709640.0
1,tt0228333,975900,/m/03vyhn,France,ghosts of mars,1937390.0
2,tt0228333,975900,/m/03vyhn,Spain,ghosts of mars,509536.0
3,tt0228333,975900,/m/03vyhn,Germany,ghosts of mars,496679.0
4,tt0228333,975900,/m/03vyhn,United Kingdom,ghosts of mars,148898.0
...,...,...,...,...,...,...
90661,tt0120202,1918494,/m/0660qx,Denmark,state and main,13732.0
90662,tt0120202,1918494,/m/0660qx,Australia,state and main,385631.0
90663,tt0107057,664006,/m/030xw6,Domestic,guilty as sin,22866222.0
90664,tt1606259,34980460,/m/0g4pl34,Domestic,knuckle,2647.0


## Question 1

For The first question we will need the gross of the movies in the different region of the world, as well as their release year and their genre.


In [25]:
movies_clean_gross = movies.copy()
gross_clean = gross.copy()

# Sum up the gross for each movie in the `gross` dataframe
gross_sum = gross_clean.groupby('Wiki_ID')['gross'].sum().reset_index()

# Merge this sum with the `movies` dataframe
movies_with_gross = movies_clean_gross.merge(gross_sum, on='Wiki_ID', how='left')

# Filter out rows where 'box_office' is not missing
non_missing_box_office = movies_with_gross.dropna(subset=['box_office'])


missing_percentage_before = movies_with_gross['box_office'].isna().sum() / len(movies_with_gross) * 100
print(f"Percentage of missing 'box_office' values before imputation: {missing_percentage_before:.2f}%")

# Check coherence for non-missing 'box_office' values
coherence_check = ((non_missing_box_office['box_office'] - non_missing_box_office['gross']).abs() / non_missing_box_office['box_office']).mean()
print(f"Average coherence deviation: {coherence_check:.2f}")

# movies_with_gross['box_office'].fillna(movies_with_gross['gross'], inplace=True) 


Percentage of missing 'box_office' values before imputation: 89.72%
Average coherence deviation: 0.34


There are a lot of box_office missing values, and for the ones we have, the summed gross differ from it by 34%, so replacing the Nans by the summed gross will not be accurate.

Instead, we should use the summed gross as the global box office for all the movies.

In [6]:
# The release date of the movie is in the format of YYYY-MM-DD, we only need the year 
# so we split the date and only keep the year
movies_clean_release = movies.copy()
movies_clean_release['release_date'] = movies_clean_release['release_date'].str[:4]
movies_clean_release['release_date'] = pd.to_numeric(movies_clean_release['release_date'], errors='coerce')
# Here the errors ignore is to keep the nan in the release date column
movies_clean_release['release_date'] = movies_clean_release['release_date'].astype(np.int64, errors='ignore')

In [7]:
movies_clean_genre = movies_clean_release.copy()
movies_clean_genre['genres'] = movies_clean_genre['genres'].apply(lambda x: list(ast.literal_eval(x).values()))
# This line is to make the empty list to be nan so the nan are uniform
movies_clean_genre['genres'] = movies_clean_genre['genres'].apply(lambda x: x if len(x) != 0 else np.nan)


In [8]:
gross_genre_release = gross.copy()
#merge gross and movies_clean_genre on wiki_ID
gross_genre_release = gross_genre_release.merge(movies_clean_genre[['Wiki_ID', 'release_date', 'genres']], on = 'Wiki_ID', how = 'left')


In [9]:
# count the numers of nan in genre
missing_genre_value = gross_genre_release.drop_duplicates(subset='Wiki_ID')['genres'].isna().sum()/gross_genre_release.drop_duplicates(subset='Wiki_ID')['genres'].shape[0]*100
print('The percentage of missing values in the genre column is {:.2f}%'.format(missing_genre_value))



The percentage of missing values in the genre column is 0.14%


We have a very low percentage of genre missing we can therefore have a good analysis about this metric. We don't have any missing values for the release date because it was used as a key when merging the IMdB dataset with the CMU datset

## Question 3 

In [10]:
movies_clean_country = movies_clean_genre.copy()
movies_clean_country['countries'] = movies_clean_country['countries'].apply(lambda x: list(ast.literal_eval(x).values()))
# This line is to make the empty list to be nan so the nan are uniform
movies_clean_country['countries'] = movies_clean_country['countries'].apply(lambda x: x if len(x) != 0 else np.nan)

In [11]:
gross_country = gross.copy()
#merge gross and movies_clean_genre on wiki_ID
gross_country = gross_country.merge(movies_clean_country[['Wiki_ID', 'countries']], on = 'Wiki_ID', how = 'left')

In [12]:
# count the numers of nan in countries
missing_country_value = gross_country.drop_duplicates(subset='Wiki_ID')['countries'].isna().sum()/gross_country.drop_duplicates(subset='Wiki_ID')['countries'].shape[0]*100
print('The percentage of missing values in the countries column is {:.2f}%'.format(missing_country_value))

The percentage of missing values in the countries column is 1.70%


In [13]:
coprood_count = gross_country.dropna(subset='countries')
coprood_count = coprood_count.drop_duplicates(subset='Wiki_ID')[coprood_count['countries'].apply(lambda x: len(x)) > 1].shape[0]/gross_country.drop_duplicates(subset='Wiki_ID')['countries'].shape[0]*100

print('The number of movies that are co-produced by more than one country is {:.2f}%'.format(coprood_count))

The number of movies that are co-produced by more than one country is 24.98%


/tmp/ipykernel_4834/30425266.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  coprood_count = coprood_count.drop_duplicates(subset='Wiki_ID')[coprood_count['countries'].apply(lambda x: len(x)) > 1].shape[0]/gross_country.drop_duplicates(subset='Wiki_ID')['countries'].shape[0]*100


We have a very low percentage of countries missing we can therefore have a good analysis about this metric. And the level of co-production is relatively high so it will still make sense to do our analysis.

## Question 4

For the gender representation in the moviee. We decided to only use the values that were given for each of the movies, making the assumption that if the gender of an actor is not given the character was probably a small part. 

In [14]:
# go in the character dataset and find the percentage of gender F in each movie
character_gender = character.copy()

#count the number of "F" for each wiki_ID in character_gender
# groupby wiki_ID and count the number of "F" in each group
character_groupby = character_gender.groupby('Wiki_ID')
character_groupby_tot = character_groupby["actor_gender"].count()
character_groupby_F = character_groupby.apply(lambda x: (x['actor_gender'] == 'F').sum())
ratio_female_to_total = character_groupby_F / character_groupby_tot
print(ratio_female_to_total)






Wiki_ID
330         1.000000
3217        0.230769
3333        0.428571
3746        0.214286
3837        0.100000
              ...   
37373877    0.500000
37476824    0.400000
37478048    0.250000
37492363         NaN
37501922    0.500000
Length: 64330, dtype: float64


In [17]:
# Analyzing Ethnicity Data
print(f"Total entries in character dataframe: {len(character)}")

# Check for missing values in 'actor_ethnicity'
missing_ethnicity = character['actor_ethnicity'].isna().sum()
print(f"Number of missing values in 'actor_ethnicity': {missing_ethnicity}")
print(f"Percentage of missing values in 'actor_ethnicity': {missing_ethnicity / len(character) * 100:.2f}%")

Total entries in character dataframe: 450669
Number of missing values in 'actor_ethnicity': 344611
Percentage of missing values in 'actor_ethnicity': 76.47%


In [19]:
#Filling Nan with 'Unknown' ethnicity
character_ethnicity = character_gender.copy()
character_ethnicity['actor_ethnicity'] = character_ethnicity['actor_ethnicity'].fillna('Unknown')

# Analyzing ethnicity distribution
ethnicity_distribution = character['actor_ethnicity'].value_counts(normalize=True) * 100
print(ethnicity_distribution)

actor_ethnicity
Unknown       76.466542
/m/0dryh9k     3.628162
/m/0x67        2.254204
/m/041rx       2.245772
/m/02w7gg      1.367966
                ...    
/m/043_z22     0.000222
/m/03x_fq7     0.000222
/m/01hphz      0.000222
/m/033fjj      0.000222
/m/013y54      0.000222
Name: proportion, Length: 480, dtype: float64


In [42]:
# Function to extract languages from the dictionary string
def extract_languages(lang_str):
    if pd.isna(lang_str):
        return ['Unknown']
    lang_dict = ast.literal_eval(lang_str)
    languages =  [lang.split(' ')[0] for lang in lang_dict.values()]
    
    return [lang for lang in languages if lang != 'Silent']


movies_clean_languages = movies_clean_country.copy()
movies_clean_languages['languages'] = movies_clean_languages['languages'].apply(extract_languages)

# Counting languages
language_counts = pd.Series([lang for sublist in movies['languages'] for lang in sublist]).value_counts()
print(language_counts)

English           40891
Hindi              3744
Spanish            3673
French             3428
Silent             3183
                  ...  
Sumerian              1
Tamang                1
Judeo-Georgian        1
Cheyenne              1
Kuna                  1
Name: count, Length: 197, dtype: int64


In [53]:
# Count the number of movies with more than one language
multilingual_movies = movies_clean_languages['languages'].apply(lambda x: len(x) > 1)

print(f"Number of multilingual movies: {multilingual_movies.sum()} over {len(multilingual_movies)} movies.")
print(movies_clean_languages['languages'][multilingual_movies].head())

Number of multilingual movies: 7207 over 81741 movies.
49                             [Afrikaans, English]
56                            [Cantonese, Standard]
58    [English, Swedish, German, Norwegian, Danish]
65                               [English, Spanish]
95                               [Italian, English]
Name: languages, dtype: object


In [52]:
# Creating a new dataframe for multilingual movies for language combination analysis
multilingual_combinations_df = movies_clean_languages.copy()
multilingual_combinations_df = multilingual_combinations_df[multilingual_combinations_df['languages'].apply(lambda x: len(x) > 1 and 'Unknown' not in x)]

# Create a distinct identifier for each multilingual language combination
multilingual_combinations_df['multilingual_combination'] = multilingual_combinations_df['languages'].apply(lambda x: '-'.join(sorted(x)))

# Count the occurrences of each multilingual language combination
multilingual_combination_counts = multilingual_combinations_df['multilingual_combination'].value_counts()
print("Most common multilingual language combinations:")
print(multilingual_combination_counts.head())  # Adjust the number of combinations to display as needed


Most common multilingual language combinations:
multilingual_combination
English-French        522
English-Spanish       506
English-Italian       278
English-German        266
Cantonese-Standard    246
Name: count, dtype: int64


In [15]:
character[character['Wiki_ID']==37501922    ]

,Wiki_ID,Freebase_ID,release_date,character_name,actor_DOB,actor_gender,actor_height,actor_ethnicity,actor_name,actor_age
429940,37501922,/m/0c0m5vt,1992,John Hunter,1966-10-11,M,1.765,NaN,Luke Perry,25.0
429941,37501922,/m/0c0m5vt,1992,Craig Murphy,1969-07-28,F,1.720,/m/041rx,Alexis Arquette,22.0
